In [10]:
pip install google-generativeai


Note: you may need to restart the kernel to use updated packages.


In [11]:
!pip install ipywidgets --upgrade


In [12]:
pip install langchain langchain-google-genai google-generativeai

Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install numexpr

Note: you may need to restart the kernel to use updated packages.


In [14]:
import os
import google.generativeai as genai
from secret_key import GOOGLE_API_KEY

In [15]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Wrap the Gemini model so LangChain understands it
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=GOOGLE_API_KEY,
    temperature=0.7
)



In [16]:
# Create prompt template
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables=['province'],
    template="Suggest the top universities in {province}"
)
prompt_template_name.format(province="Quebec")


'Suggest the top universities in Quebec'

In [17]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
response = chain.run({"province": "Alberta"})
print(response)

C:\Users\kunal\AppData\Local\Temp\ipykernel_17520\537616463.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template_name)


Alberta boasts some excellent universities.  The top two, consistently ranked highly in Canada and internationally, are:

1. **University of Alberta (U of A):** Located in Edmonton, it's the largest university in the province and a comprehensive institution with a strong research focus across many disciplines.  It's particularly well-regarded for its medical school, engineering programs, and business school.

2. **University of Calgary (U of C):** Situated in Calgary, the U of C is another leading research university with strengths in energy, health sciences, and business.  It's known for its innovative approach to teaching and research.


While these are generally considered the top two,  other universities in Alberta offer excellent programs and deserve consideration depending on specific academic interests:

* **University of Lethbridge:** A smaller, more comprehensive university with a strong focus on undergraduate education and a beautiful campus.  It excels in certain niche areas

In [18]:
from langchain.prompts import PromptTemplate
prompt_template_name = PromptTemplate(
    input_variables=['province'],
    template="Suggest the top universities in {province}?"
)

# Build LangChain LLMChain
from langchain.chains import LLMChain

name_chain =LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['University'],
    template="""Suggest some majors for {University}. Return it as a comma seperated list"""
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

In [19]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])

content = chain.run("Ontario")
print(content)

Medicine, Law, Engineering, Health Sciences, Business, Computer Science, Mathematics, Social Sciences, Humanities, Agriculture, Veterinary Medicine, Environmental Science, Psychology, Environmental Studies


In [20]:
prompt_template_name = PromptTemplate(
    input_variables=['province'],
    template="Suggest the top universities in {province}?"
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name, output_key="University")

prompt_template_name = PromptTemplate(
    input_variables=['University'],
    template="Suggest the top universities in {University}?"
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name, output_key="major")


In [21]:
from langchain.chains import LLMChain, SequentialChain
from langchain.prompts import PromptTemplate

# Step 1: Province → Major
prompt_template_major = PromptTemplate(
    input_variables=['province'],
    template="Suggest a trending computer-related major in {province}."
)
name_chain = LLMChain(llm=llm, prompt=prompt_template_major, output_key="major")  # <-- this fixes it


In [22]:

# Step 2: Major → Universities
prompt_template_universities = PromptTemplate(
    input_variables=['major'],
    template="List top universities in Canada that offer {major}. Return as a comma-separated list."
)
food_items_chain = LLMChain(llm=llm, prompt=prompt_template_universities)



In [23]:
chain = SequentialChain(
    chains=[name_chain, food_items_chain],
    input_variables=["province"],
    output_variables=["text"],  # remove "major" here
    verbose=True
)
result = chain.run({"province": "Ontario"})
print(result)




> Entering new SequentialChain chain...

> Finished chain.
University of Toronto, University of Waterloo, University of British Columbia, McGill University, McMaster University, University of Alberta, University of Calgary, Western University, Queen's University, Carleton University


In [24]:
from secret_key import GOOGLE_API_KEY

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=GOOGLE_API_KEY,
    temperature=0.7
)

response = llm.invoke("When was Elon Musk born? What is his age right now in 2025?")
print(response.content)


Elon Musk was born on June 28, 1971.  Therefore, in 2025, he is 54 years old.


In [27]:
import os
import importlib

# Import and reload in case of Jupyter cache issues
import secret_key
importlib.reload(secret_key)
from secret_key import serpapi_key, GOOGLE_API_KEY

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import AgentType, initialize_agent, load_tools

# Optional: check if key is loading
# print("Loaded SerpAPI Key:", repr(serpapi_key))

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=GOOGLE_API_KEY,
    temperature=0.7
)

tools = load_tools(
    ["serpapi", "llm-math"],
    llm=llm,
    serpapi_api_key=serpapi_key
)

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)




In [9]:
response = agent.run("What was the GDP of US in 2022 plus 5?")
print(response)

C:\Users\kunal\AppData\Local\Temp\ipykernel_17520\2164540290.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run("What was the GDP of US in 2022 plus 5?")




> Entering new AgentExecutor chain...
Thought: I need to find the US GDP for 2022, then add 5 to it.  First I'll find the GDP using a search engine.

Action: Search
Action Input: "US GDP 2022"
Observation: ['Current-dollar GDP increased 9.2 percent, or $2.15 trillion, in 2022 to a level of $25.46 trillion, compared with an increase of 10.7 percent, ...', 'U.S. GDP for 2022 was 26.007 trillion US dollars, a 9.82% increase from 2021. · U.S. GDP for 2021 was 23.681 trillion US dollars, a 10.9% increase from 2020.', 'GDP (current US$) - United States from The World Bank: Data. ... Virgin Islands (U.S.). 2022. 4,672.00. West Bank and Gaza. 2024. 13,711.10. Yemen ...', 'The Gross Domestic Product (GDP) in the United States was worth 29184.89 billion US dollars in 2024, according to official data from the World Bank.', 'A comprehensive measure of US economic activity. GDP measures the value of the final goods and services produced in the United States.', 'This pie chart shows the percentage

In [28]:
chain = LLMChain(llm=llm,prompt=prompt_template_name)
name = chain.run("Ontario")
print(name)

Defining "top" depends on the criteria used (research output, undergraduate teaching, specific programs, etc.). However, consistently ranking highly in various metrics in Ontario are:

* **University of Toronto (U of T):**  Generally considered the top university in Ontario and often among the top in Canada and globally.  Strong across all disciplines.

* **University of Waterloo:**  Excellent reputation for engineering, computer science, and mathematics.  Also known for its co-op programs.

* **McMaster University:**  Known for its innovative problem-based learning approach, particularly strong in health sciences and engineering.

* **Western University:**  Strong in business, social sciences, and health sciences.  Known for its vibrant student life.

* **Queen's University:**  Excellent reputation, particularly in business (Ivey Business School), engineering, and arts and science. Known for its smaller class sizes and strong sense of community.


Other excellent universities in Ontar

In [29]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.run("Ontario")
print(name)

C:\Users\kunal\AppData\Local\Temp\ipykernel_17520\1009817347.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


The "top" university depends on your specific criteria (e.g., research focus, program ranking, student experience), but consistently highly ranked universities in Ontario include:

* **University of Toronto (U of T):**  Generally considered the top university in Ontario and consistently ranks highly in global rankings. Strong across many disciplines, particularly in medicine, engineering, and law.

* **University of Waterloo:** Renowned for its co-op programs, particularly in engineering, computer science, and mathematics.  A very strong focus on applied research and industry collaboration.

* **McMaster University:** Known for its innovative problem-based learning approach in medicine and health sciences, but also strong in other areas like engineering and science.

* **Western University:** Strong in business, health sciences, and social sciences.  A large university with a vibrant campus life.

* **Queen's University:**  A smaller university with a strong reputation, particularly in

In [30]:
name = chain.run("Alberta")
print(name)

The top universities in Alberta, consistently ranking highly in national and international rankings, are:

* **University of Alberta (U of A):** This is generally considered the top university in Alberta, with a strong reputation across various disciplines and a large, comprehensive program offering.

* **University of Calgary (U of C):**  A strong competitor to the U of A, the University of Calgary excels in certain areas like energy research, medicine, and business.  It also has a significant research output.

While these two are clearly the top contenders, it's important to note that the "best" university depends on individual needs and preferences.  Factors like specific program strength, research opportunities, campus culture, and location all play a significant role.  Other universities in Alberta offer excellent programs in specific niches but don't usually rank as highly overall.


In [32]:
print(chain.memory.buffer)

Human: Ontario
AI: The "top" university depends on your specific criteria (e.g., research focus, program ranking, student experience), but consistently highly ranked universities in Ontario include:

* **University of Toronto (U of T):**  Generally considered the top university in Ontario and consistently ranks highly in global rankings. Strong across many disciplines, particularly in medicine, engineering, and law.

* **University of Waterloo:** Renowned for its co-op programs, particularly in engineering, computer science, and mathematics.  A very strong focus on applied research and industry collaboration.

* **McMaster University:** Known for its innovative problem-based learning approach in medicine and health sciences, but also strong in other areas like engineering and science.

* **Western University:** Strong in business, health sciences, and social sciences.  A large university with a vibrant campus life.

* **Queen's University:**  A smaller university with a strong reputati